In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

17465344/17464789 [==============================] - 28s 2us/step


In [4]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [5]:
data = vectorize(data)
targets = np.array(targets).astype("float32")
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [6]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
_________________________________________________________________


In [7]:
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [10]:
results = model.fit(
 train_x, train_y,
 epochs= 10,
 batch_size = 500,
 validation_data = (test_x, test_y)
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 22s 545us/step - loss: 0.0515 - acc: 0.9831 - val_loss: 0.4524 - val_acc: 0.8827
Epoch 2/10
40000/40000 [==============================] - 21s 518us/step - loss: 0.0417 - acc: 0.9857 - val_loss: 0.5106 - val_acc: 0.8811
Epoch 3/10
40000/40000 [==============================] - 17s 436us/step - loss: 0.0349 - acc: 0.9881 - val_loss: 0.5312 - val_acc: 0.8806
Epoch 4/10
40000/40000 [==============================] - 18s 458us/step - loss: 0.0352 - acc: 0.9876 - val_loss: 0.5289 - val_acc: 0.8821
Epoch 5/10
40000/40000 [==============================] - 17s 435us/step - loss: 0.0298 - acc: 0.9897 - val_loss: 0.5704 - val_acc: 0.8817
Epoch 6/10
40000/40000 [==============================] - 16s 388us/step - loss: 0.0245 - acc: 0.9919 - val_loss: 0.6289 - val_acc: 0.8776
Epoch 7/10
40000/40000 [==============================] - 14s 359us/step - loss: 0.0243 - acc: 0.9907 - val_loss: 0.

In [11]:
model.save("my_model.h5")

https://towardsdatascience.com/how-to-build-a-neural-network-with-keras-e8faa33d0ae4